This notebook is for automating ddl generation for csv files

In [2]:
import pandas as pd
from sqlalchemy import create_engine, text
import io
import os

In [3]:
# pip install sqlalchemy

In [5]:
def get_filename(file_path):
    """
    Extracts the filename without its extension from a given file path.

    Args:
        file_path (str): The full path to the file (e.g., 'C:\\Users\\dt\\Downloads\\E-commerce Mock data\\customers.csv').

    Returns:
        str: The filename without the extension (e.g., 'payments').
    """

    base_name = os.path.basename(file_path)

    # Step 2: Split the base name into root and extension
    # os.path.splitext separates 'payments.csv' into ('payments', '.csv')
    file_name_without_extension, _ = os.path.splitext(base_name)

    return file_name_without_extension

In [6]:
# --- Configuration ---
CSV_FILEPATH = r'C:\Users\jemima.villanueva_st\Documents\strat-codebase\data-engineering-bootcamp\datasets\ecommerce_data\customers.csv'  # <--- Change this to your CSV file name
TABLE_NAME = get_filename(CSV_FILEPATH)  # <--- Change this to your desired table name
SQL_OUTPUT_DIR = r'C:\Users\jemima.villanueva_st\Documents\strat-codebase\data-engineering-bootcamp\sql_scripts\scripts' #  <--- Change this to your actual path for the output
DDL_OUTPUT_FILENAME = f'create_{TABLE_NAME}_table.sql'

In [7]:
# --- 1. Read CSV into Pandas DataFrame ---
try:
    df = pd.read_csv(CSV_FILEPATH)
    print(f"Successfully loaded '{CSV_FILEPATH}' with {len(df)} rows and {len(df.columns)} columns.")
except FileNotFoundError:
    print(f"Error: CSV file '{CSV_FILEPATH}' not found. Make sure it's in the correct directory.")
    exit()
except Exception as e:
    print(f"Error reading CSV: {e}")
    exit()

# --- 2. Map the data types and generate the DDL script ---
dtype_mapping = {
    'object': 'TEXT',
    'int64': 'BIGINT',
    'float64': 'NUMERIC',
    'bool': 'BOOLEAN',
    'datetime64[ns]': 'TIMESTAMP',
    # Add more mappings as needed
}

column_defs = []
for col_name, dtype in df.dtypes.items():
    sql_type = dtype_mapping.get(str(dtype), 'TEXT') # Default to TEXT if not in map
    # Handle potential spaces or special characters in column names
    safe_col_name = f'"{col_name.replace(" ", "_").lower()}"'
    column_defs.append(f"{safe_col_name} {sql_type}")

ddl_script = f"CREATE TABLE IF NOT EXISTS {TABLE_NAME} (\n    " + ",\n    ".join(column_defs) + "\n);"
print("\n--- Generated DDL Script ---")
print(ddl_script)
print("---------------------------\n")


Successfully loaded 'C:\Users\jemima.villanueva_st\Documents\strat-codebase\data-engineering-bootcamp\datasets\ecommerce_data\customers.csv' with 10 rows and 6 columns.

--- Generated DDL Script ---
CREATE TABLE IF NOT EXISTS customers (
    "customer_id" TEXT,
    "name" TEXT,
    "email" TEXT,
    "registration_date" TEXT,
    "city" TEXT,
    "country" TEXT
);
---------------------------



In [8]:
# --- 3. Save DDL Script to File ---
output_full_path = os.path.join(SQL_OUTPUT_DIR, DDL_OUTPUT_FILENAME)

try:
    # Create the output directory if it doesn't exist
    os.makedirs(SQL_OUTPUT_DIR, exist_ok=True)

    with open(output_full_path, 'w') as f:
        f.write(ddl_script)
    print(f"DDL script saved successfully to: '{output_full_path}'")
except Exception as e:
    print(f"Error saving DDL script to file: {e}")

DDL script saved successfully to: 'C:\Users\jemima.villanueva_st\Documents\strat-codebase\data-engineering-bootcamp\sql_scripts\scripts\create_customers_table.sql'
